In [15]:
import pandas as pd
import calendar
import numpy as np
import random
from statistics import mode
import os
import warnings
import time
from datetime import datetime

warnings.filterwarnings("ignore")

hacer séptimo mes

In [16]:
path_daily_data = "C:/Users/lllanos/py_notebooks/example_workshop/ethiopia/inputs/climatePrediction/dailyData/"
path_probabilities = "C:/Users/lllanos/py_notebooks/example_workshop/ethiopia/inputs/"
path_output = "C:/Users/lllanos/py_notebooks/example_workshop/ethiopia/outputs/climatePrediction/resampling"
date_start = '2023-11-01' # Date of the first month of the season to forecast
year_forecast = 2023 # Year of the first month of the season to forecast

In [17]:
from shutil import rmtree
if os.path.exists(path_output):
    rmtree(path_output)

In [18]:
def mdl_verification(daily_weather_data, seasonal_probabilities):
    #################################
    ####  Clima y Probabilidad   ####
    #################################

    clima = os.listdir(daily_weather_data)
    clima = [file for file in clima if not file.endswith("_coords.csv")]
    clima = [file.split(".csv")[0] for file in clima]

    prob = pd.read_csv(os.path.join(seasonal_probabilities , "probabilities.csv"))

    ###############################################################
    ### Tener en cuenta solo los Ids de clima en la base Prob   ###
    ###############################################################

    prob = prob[prob['id'].isin(clima)]

    #######################
    ### Revisemos clima ###
    #######################

    check_clm = []
    for i in range(len(clima)):
        df = pd.read_csv(os.path.join(daily_weather_data, f"{clima[i]}.csv"))

        # 1. max de temp_max == min de temp_max
        # 2. max de temp_min == min de temp_min
        # 3. max de srad == min de srad

        max_tmax = df['t_max'].max()
        min_tmax = df['t_max'].min()

        max_tmin = df['t_min'].max()
        min_tmin = df['t_min'].min()

        max_srad = df['sol_rad'].max()
        min_srad = df['sol_rad'].min()

        if max_tmax == min_tmax or max_tmin == min_tmin or max_srad == min_srad:
            resultado = pd.DataFrame({'code': [clima[i]], 'value': [f"tmax = {max_tmax}; tmin = {max_tmin}; srad = {max_srad}"]})
        else:
            resultado = pd.DataFrame({'code': [clima[i]], 'value': ["OK"]})
        check_clm.append(resultado)

    df = pd.concat(check_clm)
    df_1 = df[df['value'] == "OK"]
    df_2 = df[df['value'] != "OK"]

    code_ok = df_1
    code_problema = df_2

    ##################################
    ####  Revisemos probabilidad  ####
    ##################################

    # 1. Probabilidades con cero categoria normal
    # 2. Probabilidades sumen > 1.1
    # 3. Probabilidades sumen <  0.9

    prob['sum'] = prob['below'] + prob['normal'] + prob['above']
    prob.loc[prob['normal'] == 0.00, 'normal'] = -1
    prob.loc[prob['sum'] < 0.9, 'normal'] = -1
    prob.loc[prob['sum'] > 1.1, 'normal'] = -1

    df_1 = prob[prob['normal'] == -1]
    df_2 = prob[(prob['normal'] >= 0) & (prob['normal'] <= 1)]

    code_p_ok = df_2
    code_p_malos = df_1


    # Ids buenos: Datos de clima y probabilidades sin problemas
    ids_buenos = code_p_ok['id'].tolist()

    # Ids malos: clima malo - prob mala - outside
    result_clima_prob_outside = list(set(clima) - set(code_p_ok['id']))

    code_problema = pd.DataFrame({'ids':code_problema['code'].tolist(),
                                  'descripcion': code_problema['value'].tolist()})
    code_p_malos = pd.DataFrame({'ids': code_p_malos['id'].tolist(),
                                 'descripcion': "Problemas base de datos probabilidad"})

    result_clima_prob_outside = pd.DataFrame({'ids': result_clima_prob_outside,
                                              'descripcion': "La estacion esta fuera de area predictora"})


    ids_malos = pd.concat([code_problema, code_p_malos, result_clima_prob_outside])
    ids_buenos = pd.DataFrame({'ids': ids_buenos})

    ids_malos = ids_malos.replace(1, pd.NA).dropna()

    result = {'ids_buenos': ids_buenos, 'ids_malos': ids_malos}
    return result

In [19]:
def preprocessing(prob_root,  ids):

    """ Determine seasons of analysis according to the month of forecast in CPT

    Args:

    prob_root: str
              The root of the probabilities file from CPT, with its name and extension.


    ids: dict
              Dictionary with a list of stations with problems and not to be analyzed, and
              a list of stations without problems.

    Returns:

      Dataframe
          a dataframe with the original columns + name of season + start month of season +
          end month of season

    """


    # Read the CPT probabilities file

    proba = pd.read_csv(os.path.join(prob_root , "probabilities.csv"))

    ids_x = ids['ids_buenos']
    prob = proba[proba['id'].isin(ids_x['ids'])]
    s = prob['season'].iloc[0]
    forecast_period = "tri" if s.count("-") > 1 else "bi"

    # Check the period of forecast
    if forecast_period == "tri":

      # Create a list of month numbers from 1 to 12, followed by [1, 2] to create quarters
      months_numbers =list(range(1,13)) + [1,2]

      # Create a DataFrame representing periods of three consecutive months (with its numbers)
      period= pd.DataFrame( [months_numbers[i:i+3] for i in range(0, len(months_numbers)-2)])
      period.columns = ['Start', 'Central_month', 'End']


      # Merge the prob DataFrame with the period DataFrame based on the 'month' and 'Central_month' columns
      prob = prob.merge(period, left_on='month', right_on='Central_month')
      prob.drop(['month','Central_month'], axis = 1, inplace = True )

    else:
      if forecast_period == "bi":

        # Create a list of month numbers from 1 to 12
        months_numbers = list(range(1,13))
        months_numbers.append(1)


        # Create a DataFrame representing periods of two consecutive months (with its numbers)
        period = pd.DataFrame( [months_numbers[i:i+2] for i in range(0, len(months_numbers)-1)])
        period.columns = ['Start', 'End']


        # Merge the prob DataFrame with the period DataFrame based on the 'month' and 'Start' month columns
        prob = prob.merge(period, left_on='month', right_on='Start')

        # Merge the prob DataFrame with the period DataFrame based on the 'month' and 'End' month columns
        # Join with prob_a
        #prob = prob_a.append(prob.merge(period, left_on='month', right_on='End'))
        prob.drop(['month'], axis = 1, inplace = True )

    # Reshape the 'prob' DataFrame and put the 'below', 'normal' and 'above' probability categories in a column
    prob = prob.melt(id_vars = ['year', 'id', 'predictand','season', 'Start','End'], var_name = 'Type', value_name = 'Prob')

    #Return probability DataFrame
    return prob, forecast_period

In [20]:
def forecast_station(station, prob, daily_data_root, output_root, year_forecast, forecast_period):

    """ Generate  forecast scenaries

    Args:

    station: str
            The id of th station

      prob: DataFrame
              The result of preprocessing function

      daily_data_root: str
              Where the climate data by station is located

      output_root: str
              Where outputs are going to be saved.

      year_forecast: int
              Year to forecast

      forecast_period: str
              'bi' if the period of CPT forecast is bimonthly.
              'tri' if the period of CPT forecast is quarter.

    Returns:

      Dataframe
          a dataframe with climate daily data for every season and escenary id
          a dataframe with years of escenary for every season

    """
    # Create folders to save result
    val_root = os.path.join(output_root, "validation")
    if not os.path.exists(val_root):
        os.makedirs(val_root)

    # Read the climate data for the station
    clim = pd.read_csv(os.path.join(daily_data_root ,f"{station}.csv"))

    # Filter the probability data for the station
    cpt_prob = prob[prob['id']==station]

    if len(cpt_prob.index) == 0:
      print('Station does not have probabilites')
      base_years = 0
      seasons_range = 0
      p = {'id': [station],'issue': ['Station does not have probabilites']}
      problem = pd.DataFrame(p)

      return base_years, seasons_range,  problem

    else:
      # Get the season for the forecast
      season = np.unique(cpt_prob['season'])
      tri_seasons = ['Dec-Jan-Feb', 'Jan-Feb-Mar', 'Feb-Mar-Apr', 'Jan-Feb', 'Feb-Mar']

      # Adjust the year if the forecast period is 'tri' if necessary
      if  any(np.isin(season, tri_seasons)) :
         year_forecast = year_forecast+1

      # Check if year of forecast is a leap year for February
      leap_forecast = (year_forecast%400 == 0) or (year_forecast%4==0 and year_forecast%100!=0)

      # Filter the February data for leap years
      clim_feb = clim.loc[clim['month'] == 2]
      clim_feb['leap'] = [True if (year%400 == 0) or (year%4==0 and year%100!=0) else False for year in clim_feb['year']]

      # Standardize february months by year according to year of forecat
      february = pd.DataFrame()
      for i in np.unique(clim_feb['year']):
        year_data =  clim_feb.loc[clim_feb['year']==i,:]
        year = year_data.loc[:,'leap']

        # If year of forecast is a leap year and a year in climate data is not, then add one day to february in climate data
        if leap_forecast == True and year.iloc[0] == False:
          year_data = pd.concat([year_data, year_data.sample(1)], ignore_index=True)
          year_data.iloc[-1,0] = 29
        else:

          # If year of forecast is not a leap year and a year in climate data is, then remove one day to february in climate data
          if leap_forecast == False and year.iloc[0] == True:
            year_data =  year_data.iloc[:-1]
          else:

            # If both year of forecast and year in climate data are leap years or not, then keep climate data the same
            year_data = year_data
        february = pd.concat([february, year_data])


      # Concat standardized february data with the rest of climate data
      data = february.drop(['leap'], axis = 1 )
      data = pd.concat([data,clim.loc[clim['month'] != 2]]).sort_values(['year','month'])

      # Start the resampling process for every season of analysis in CPT probabilities file

      base_years =  pd.DataFrame() # List to store years of sample for each season
      seasons_range = pd.DataFrame() # List to store climate data in the years of sample for each season

      for season in  list(np.unique(cpt_prob['season'])):

        # Select the probabilities for the season
        x = cpt_prob[cpt_prob['season'] == season]


        predictand = cpt_prob['predictand'].iloc[0]


      # Compute total precipitation for each year in the climate data range selected
        new_data = data[['year',predictand]].groupby(['year']).sum().reset_index()

        data['season'] = season


      # Calculate quantiles to determine precipitation conditions for every year in climate data selected
        cuantiles = list(np.quantile(new_data['prec'], [.33,.66]))
        new_data['condition'] =  'NA'
        new_data.loc[new_data[predictand]<= cuantiles[0], 'condition'] = 'below'
        new_data.loc[new_data[predictand]>= cuantiles[1], 'condition'] =  'above'
        new_data.loc[(new_data[predictand]> cuantiles[0]) & (new_data[predictand]< cuantiles[1]), 'condition'] =  'normal'

      # Sample 100 records in probability file of season based on probability from CPT as weights
        muestras = x[['Start', 'End', 'Type', 'Prob']].sample(100, replace = True, weights=x['Prob'])
        muestras = muestras.set_index(pd.Index(list(range(0,100))))

      # Randomly get one year from the total precipitation data based on precipitation conditions selected in the 100 data sample.
        muestras_by_type = []
        for i in muestras.index:
          m = new_data.loc[new_data['condition'] == muestras['Type'].iloc[i]].sample(1)

          if any(m['year'] == max(new_data['year'])):
            b = new_data.loc[new_data['condition'] == muestras['Type'].iloc[i]]
            m = b[b['year'] != max(new_data['year'])].sample(1)
          else:
            m = m

          muestras_by_type.append(m)

        # Join the 100 samples and add sample id
        muestras_by_type = pd.concat(muestras_by_type).reset_index()
        muestras_by_type['index'] = muestras.index
        #muestras_by_type = muestras_by_type.set_index(pd.Index(list(range(0,100))))


        # Rename year column with season name
        muestras_by_type = muestras_by_type.rename(columns = {'year':season})

        #Set the sample years as list and sort
        years = list(muestras_by_type[season])
        years.sort()


        if season == 'Nov-Dec-Jan':
          # If season is November-December-January

          # Calculate the next year of the year sample and assign the same sample id
          muestras_by_type['plus'] = list(map(lambda x: x + 1, muestras_by_type[season]))


          years_plus = list(map(lambda x: x + 1, years))
          years_plus.sort()

          # Filter the climate data of the last two months of the years in the sample and get the sample id
          merge_a =  data[data['year'].isin(years)]
          merge_a = merge_a[merge_a['month'].isin([11,12])]
          merge_a = pd.merge(merge_a, muestras_by_type[['index', season]], left_on = 'year', right_on = season)
          merge_a.drop(season, axis = 1,inplace = True)

          # Filter the climate data of the first month in the next year of the years in sample and get the sample id
          merge_b = data[data['year'].isin(years_plus)]
          merge_b = merge_b[merge_b['month'] == 1]
          merge_b = pd.merge(merge_b, muestras_by_type[['index', 'plus']], left_on = 'year', right_on = 'plus')
          merge_b.drop('plus', axis = 1,inplace = True)

          # Merge the climate data filtered
          merge = pd.concat([merge_a, merge_b])


        else:
          if season == 'Dec-Jan-Feb':
            # If season is December-January-February


            # Calculate the next year of the year sample and assign the same sample id
            muestras_by_type['plus'] = list(map(lambda x: x + 1, muestras_by_type[season]))

            years_plus = list(map(lambda x: x + 1, years))
            years_plus.sort()

            # Filter the climate data of the last month of the years in the sample and get the sample id

            merge_a = data[data['year'].isin(years)]
            merge_a = merge_a[merge_a['month'] == 12]
            merge_a = pd.merge(merge_a, muestras_by_type[['index', season]], left_on = 'year', right_on = season)
            merge_a = merge_a.drop(columns = [season])

            # Filter the climate data of the first two months in the next year of the years in sample and get the sample id

            merge_b = data[data['year'].isin(years_plus)]
            merge_b = merge_b[merge_b['month'].isin([1,2])]
            merge_b = pd.merge(merge_b, muestras_by_type[['index', 'plus']], left_on = 'year', right_on = 'plus')
            merge_b = merge_b.drop(columns = ['plus'])

            # Merge filtered data
            merge = pd.concat([merge_a, merge_b])


          else:
            if season == 'Dec-Jan':

                    # Calculate the next year of the year sample and assign the same sample id
                    muestras_by_type['plus'] = list(map(lambda x: x + 1, muestras_by_type[season]))

                    years_plus = list(map(lambda x: x + 1, years))
                    years_plus.sort()

                    # Filter the climate data of the last month of the years in the sample and get the sample id

                    merge_a = data[data['year'].isin(years)]
                    merge_a = merge_a[merge_a['month'] == 12]
                    merge_a = pd.merge(merge_a, muestras_by_type[['index', season]], left_on = 'year', right_on = season)
                    merge_a = merge_a.drop(columns = [season])

                    # Filter the climate data of the first two months in the next year of the years in sample and get the sample id

                    merge_b = data[data['year'].isin(years_plus)]
                    merge_b = merge_b[merge_b['month'] == 1]
                    merge_b = pd.merge(merge_b, muestras_by_type[['index', 'plus']], left_on = 'year', right_on = 'plus')
                    merge_b = merge_b.drop(columns = ['plus'])

                    # Merge filtered data
                    merge = pd.concat([merge_a, merge_b])

            else:
                    # If season is another, filter climate data of the years in sample and get the sample id

                    merge = data.loc[data['year'].isin(years)]
                    merge = merge.loc[(merge['month'] >= x['Start'].iloc[0]) & (merge['month'] <= x['End'].iloc[0])]
                    merge = pd.merge(merge,muestras_by_type[['index',season]],left_on = 'year', right_on = season)
                    merge = merge.drop(columns = [season])


        # Join seasons samples by column by sample id
        base_years = pd.concat([base_years, muestras_by_type[['index',season]]], axis = 1,ignore_index=True)

        # Join climate data filtered for the seasons
        seasons_range = pd.concat([seasons_range, merge])

      seasons_range = seasons_range.rename(columns = {'index': 'id'})

      if (forecast_period == 'tri') and (len(list(np.unique(cpt_prob['season']))) == 2):

            s = list(np.unique(cpt_prob['season']))
            base_years = base_years.iloc[:,[0,1,3] ]
            base_years = base_years.rename(columns={0: 'id',1: s[0], 3: s[1]})
            base_years['id'] = base_years['id'] + 1
            seasons_range['id'] = seasons_range['id']+1
            seasons_range = seasons_range.sort_values(by=['year', 'month'], ascending=True)
            base_years.to_csv(os.path.join(val_root,  f"{station}_Escenario_A.csv"), index = False)


            #Return climate data filtered with sample id
            return base_years, seasons_range

      else:
          if (forecast_period == 'bi') and (len(list(np.unique(cpt_prob['season']))) == 3) :

            s = list(np.unique(cpt_prob['season']))
            base_years = base_years.iloc[:,[0,1,3,5] ]
            base_years = base_years.rename(columns={0: 'id',1: s[0], 3: s[1], 5: s[2]})
            base_years['id'] = base_years['id'] + 1
            seasons_range['id'] = seasons_range['id']+1
            seasons_range = seasons_range.sort_values(by=['year', 'month'], ascending=True)
            base_years.to_csv(os.path.join(val_root,  f"{station}_Escenario_A.csv"), index = False)


            #Return climate data filtered with sample id
            return base_years, seasons_range

          else:

            print('Station does not have all the seasons availables')

            s = list(np.unique(cpt_prob['season']))
            if len(base_years.columns) == 2:
              base_years = base_years.iloc[:,[0,1] ]
              base_years = base_years.rename(columns={0: 'id',1: s[0]})
            else:
              if len(base_years.columns == 4):
                base_years = base_years.rename(columns={0: 'id',1: s[0], 3: s[1]})
              else:
                base_years = base_years.rename(columns={0: 'id',1: s[0]})

            base_years['id'] = base_years['id'] + 1
            seasons_range['id'] = seasons_range['id']+1


            p = {'id': [station],'issue': ['Station does not have all the seasons availables'], 'Seasons available': ", ".join([str(item) for item in s])}
            problem = pd.DataFrame(p)
            print(problem)
            base_years.to_csv(os.path.join(val_root, f"{station}_Escenario_A.csv"), index = False)

            #Return climate data filtered with sample id
            return base_years, seasons_range, problem

In [21]:
def add_year(year_forecast, m, date_start):
  date_start = datetime.strptime(date_start, "%Y-%m-%d") #convertir a fecha
  if m < date_start.month:
    a = year_forecast + 1
  else:
    a = year_forecast

  return a

In [22]:
def save_forecast(station, output_root, year_forecast, seasons_range, base_years, date_start):


    if isinstance(base_years, pd.DataFrame):
    # Set the output root based on forecast period
      output_estacion = os.path.join(output_root, station)
      if not os.path.exists(output_estacion):
          os.makedirs(output_estacion)
          print("Path created for the station: {}".format(station))

      output_summary = os.path.join(output_root, "summary")
      if not os.path.exists(output_summary):
          os.makedirs(output_summary)

      escenarios = []
      IDs= list(np.unique(seasons_range['id']))
      seasons =  list(np.unique(seasons_range['season']))
      year_forecast = int(year_forecast)

      for i in range(len(IDs)):
        

        df = seasons_range[(seasons_range['id'] == IDs[i])]

        df = df.reset_index()
        df = df.drop(columns = ['year'])
        for j in list(range(len(df))):

            df.loc[j, 'year'] = add_year(year_forecast, df.loc[j, 'month'], date_start)

        df = df.drop(['index','id', 'season'], axis = 1)
        df['year'] = df['year'].astype('int')

        escenarios.append(df)
        df.to_csv(os.path.join(output_estacion ,f"{station}_escenario_{str(i+1)}.csv"), index=False)

      print("Escenaries saved in {}".format(output_estacion))

      # Calculate maximum and minimum of escenaries by date and save
      df = pd.concat(escenarios)
      columns = list(df.columns)
      columns.remove('year')
      new_columns = columns[:2] + ['year'] + columns[2:]
      df = df[new_columns]


      df.groupby(['year', 'month', 'day']).max().reset_index().sort_values(['month', 'day'], ascending = True).to_csv(os.path.join(output_summary, f"{station}_escenario_max.csv"), index=False)
      df.groupby(['year', 'month', 'day']).min().reset_index().sort_values(['month', 'day'], ascending = True).to_csv(os.path.join(output_summary, f"{station}_escenario_min.csv"), index=False)
      print("Minimum and Maximum of escenaries saved in {}".format(output_summary))

      vars = df.columns
      vars = [item for item in vars if item != "year"]
      vars = [item for item in vars if item != "month"]
      vars = [item for item in vars if item != "day"]

      for i in range(len(vars)):
         print(df.groupby(['year', 'month'])[vars[i]].mean().reset_index().rename(columns = {vars[i]: 'avg'}).sort_values(['year', 'month'], ascending = True))

         df.groupby(['year', 'month'])[vars[i]].max().reset_index().rename(columns = {vars[i]: 'max'}).sort_values(['year', 'month'], ascending = True).to_csv(os.path.join(output_summary, f"{station}_{vars[i]}_max.csv"), index=False)
         df.groupby(['year', 'month'])[vars[i]].min().reset_index().rename(columns = {vars[i]: 'min'}).sort_values(['year', 'month'], ascending = True).to_csv(os.path.join(output_summary, f"{station}_{vars[i]}_min.csv"), index=False)
         df.groupby(['year', 'month'])[vars[i]].mean().reset_index().rename(columns = {vars[i]: 'avg'}).sort_values(['year', 'month'], ascending = True).to_csv(os.path.join(output_summary, f"{station}_{vars[i]}_avg.csv"), index=False)


      print("Minimum, Maximum and Average of variables by escenary is saved in {}".format(output_summary))
      return df


    else:

      return None

In [23]:
def resampling_master(station, verifica,input_root, climate_data_root,  output_root, year_forecast, date_start):

    if not os.path.exists(output_root):
        os.makedirs(output_root)
        print("Path created for outputs")

    print("Processing station: " + str(station))

    print("Reading the probability file and getting the forecast seasons")
    prob_normalized = preprocessing(input_root, verifica)


    print("Resampling and creating the forecast scenaries")
    resampling_forecast = forecast_station(station = station,
                                           prob = prob_normalized[0],
                                           daily_data_root = climate_data_root,
                                           output_root = output_root,
                                           year_forecast = year_forecast,
                                           forecast_period= prob_normalized[1])


    print("Saving escenaries and a summary")
    save_forecast(station = station,
                  output_root = output_root,
                  year_forecast = year_forecast,
                  base_years = resampling_forecast[0],
                  seasons_range = resampling_forecast[1],
                  date_start = date_start)

    if len(resampling_forecast) == 3:
        oth =os.path.join(output_root, "issues.csv")
        resampling_forecast[2].to_csv(oth, mode='a', index=False, header=not os.path.exists(oth))

    else:
        return None

To run just one station:

In [24]:
estaciones = os.listdir(path_daily_data)
n = [i for i in estaciones if not  i.endswith("_coords.csv") ]
n = [i.replace(".csv","") for i in n]

verifica = mdl_verification(path_daily_data, path_probabilities)


In [25]:
resampling_master(station =n[0],
                  input_root =  path_probabilities,
                  climate_data_root = path_daily_data,
                  output_root = path_output,
                  verifica = verifica,
                  year_forecast = year_forecast,
                  date_start = date_start )


Path created for outputs
Processing station: woyalita
Reading the probability file and getting the forecast seasons
Resampling and creating the forecast scenaries
Saving escenaries and a summary
Path created for the station: woyalita
Escenaries saved in C:/Users/lllanos/py_notebooks/example_workshop/ethiopia/outputs/climatePrediction/resampling\woyalita
Minimum and Maximum of escenaries saved in C:/Users/lllanos/py_notebooks/example_workshop/ethiopia/outputs/climatePrediction/resampling\summary
   year  month        avg
0  2023     11  23.395150
1  2023     12  24.200668
2  2024      1  24.846794
3  2024      2  25.629779
4  2024      3  25.331013
5  2024      4  23.782743
   year  month        avg
0  2023     11  13.237603
1  2023     12  13.244232
2  2024      1  13.791277
3  2024      2  14.225431
4  2024      3  14.779426
5  2024      4  14.588327
   year  month       avg
0  2023     11  1.954133
1  2023     12  0.683058
2  2024      1  0.962981
3  2024      2  1.467224
4  2024    